In [0]:
import numpy as np
import seaborn as sns
import scipy
import pandas as pd
from sklearn.datasets import make_classification

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data ,target = make_classification(n_samples = 1000 , n_features = 10 , n_clusters_per_class = 1 , n_classes = 3) 

In [0]:
train_data , train_label = data[:700] , target[:700]
test_data , test_label = data[300 :] , target[300 : ]
new = pd.DataFrame(data)
train_data , train_label =  np.array(train_data) , np.array(train_label)
test_data , test_label =  np.array(test_data) , np.array(test_label)
train_label = np.array(pd.get_dummies(train_label))
test_label = np.array(pd.get_dummies(test_label))

In [0]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def sigmoid_derv(s):
    return s * (1 - s)

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss


In [0]:
class MyNN:
    def __init__(self, x, y):
        self.x = x
        neurons = 128
        self.lr = 0.5
        ip_dim = x.shape[1]
        op_dim = y.shape[1]

        self.w1 = np.random.randn(ip_dim, neurons)
        self.b1 = np.zeros((1, neurons))
        self.w2 = np.random.randn(neurons, neurons)
        self.b2 = np.zeros((1, neurons))
        self.w3 = np.random.randn(neurons, op_dim)
        self.b3 = np.zeros((1, op_dim))
        self.y = y

    def feedforward(self):
        z1 = np.dot(self.x, self.w1) + self.b1
        self.a1 = sigmoid(z1)
        z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(z2)
        z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = softmax(z3)
        
    def backprop(self):
        loss = error(self.a3, self.y)
        print('Error :', loss)
        a3_delta = cross_entropy(self.a3, self.y) # w3
        z2_delta = np.dot(a3_delta, self.w3.T)
        a2_delta = z2_delta * sigmoid_derv(self.a2) # w2
        z1_delta = np.dot(a2_delta, self.w2.T)
        a1_delta = z1_delta * sigmoid_derv(self.a1) # w1

        self.w3 -= self.lr * np.dot(self.a2.T, a3_delta)
        self.b3 -= self.lr * np.sum(a3_delta, axis=0, keepdims=True)
        self.w2 -= self.lr * np.dot(self.a1.T, a2_delta)
        self.b2 -= self.lr * np.sum(a2_delta, axis=0)
        self.w1 -= self.lr * np.dot(self.x.T, a1_delta)
        self.b1 -= self.lr * np.sum(a1_delta, axis=0)

    def predict(self, data):
        self.x = data
        self.feedforward()
        return self.a3.argmax()

In [0]:
model = MyNN(train_data , train_label)


In [54]:
epochs = 1500
for x in range(epochs):
    model.feedforward()
    model.backprop()
		
def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = model.predict(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100
	
print("Training accuracy : ", get_acc(train_data , train_label))
print("Test accuracy : ", get_acc(test_data , test_label))

Error : 6.491281828067044
Error : 4.933979448507053
Error : 2.9178001045915694
Error : 4.44990238115093
Error : 2.037188222823285
Error : 2.0240329164560458
Error : 2.0486537273599907
Error : 1.5799224717241867
Error : 1.0135422183492104
Error : 0.8632449829979625
Error : 0.7063664935668772
Error : 0.6212263760880632
Error : 0.5674714796970282
Error : 0.5300878186062811
Error : 0.5045710506400055
Error : 0.49166597778483817
Error : 0.48454684274976484
Error : 0.4909351487709394
Error : 0.5026441366262209
Error : 0.5259070915077143
Error : 0.5531105111186531
Error : 0.5765448114721416
Error : 0.6015555238907254
Error : 0.6001896600525707
Error : 0.6035021008215158
Error : 0.5737001708874191
Error : 0.5546669268786906
Error : 0.5164557327609216
Error : 0.49252506910560806
Error : 0.4634532038120999
Error : 0.4484528955452165
Error : 0.4307320981244828
Error : 0.4263364258837836
Error : 0.4158407638591031
Error : 0.4193919838529278
Error : 0.4120100732549218
Error : 0.4204505763027482
Err